# Detección de spam

In [1]:
import numpy
import pandas
import random

Primero se encuentra el porcentaje de correos que son spam y los que no lo son.

In [2]:
datacsv = pandas.read_csv('spam.csv', sep=' ')
datanp = datacsv.to_numpy()
num_data = datanp.shape[0] # número total de datos (de filas)
num_vars = datanp.shape[1]-1 # número total de variables (columnas-1)
num_spam = 0
for x in datanp:
    if x[num_vars] == 1:
        num_spam += 1
print("Correos spam:", str(100*(num_spam/num_data))+'%')
print("Correos no spam:", str(100*(num_data-num_spam)/num_data)+'%')

Correos spam: 28.98859021465867%
Correos no spam: 71.01140978534133%


Ahora se va a partir el conjunto de datos en 3:
- El de entrenamiento, que tendrá el 60% de los datos.
- El de validación, que tendrá el 20%.
- El de prueba, que tendrá el otro 20%.

In [3]:
# Parte al conjunto de datos en 3, de forma aleatoria.
# Cada parámetro (entrenamiento, validación y prueba) es una fracción del tamaño del
# subconjunto de datos entre el tamaño del conjunto total.
def particion(datos, entrenamiento, validacion, prueba):
    n = datos.shape[0]
    indices_total = [i for i in range(n)]
    indices_ent = []
    random.seed(0) #Se usa 0 como semilla
    while (len(indices_ent)/n) < entrenamiento:
        indice_rand = random.randint(0, len(indices_total)-1)
        indices_ent.append(indices_total[indice_rand])
        del indices_total[indice_rand]
    indices_val = []
    while (len(indices_val)/n) < validacion:
        indice_rand = random.randint(0, len(indices_total)-1)
        indices_val.append(indices_total[indice_rand])
        del indices_total[indice_rand]
    #En este punto, indices_total tendrá los índices para validación
    datos_ent = []
    datos_val = []
    datos_pru = []
    for i in indices_ent:
        datos_ent.append(datos[i])
    for i in indices_val:
        datos_val.append(datos[i])
    for i in indices_total:
        datos_pru.append(datos[i])
    return (numpy.array(datos_ent), numpy.array(datos_val), numpy.array(datos_pru))

data_ent, data_val, data_pru = particion(datanp, 0.6, 0.2, 0.2)

El primer clasificador utilizará distribución de Bernoulli, donde un número mayor a 0 indica la presencia de cierta palabra y 0 indica ausencia de esa palabra.

In [4]:
# Distribución de bernoulli
def bernoulli(x, q):
    return q**x * (1.0 - q)**(1.0 - x)

In [5]:
class BernoulliNBLog:
    ## Estima parámetros por máxima verosimilitud
    def fit(self, X, y):
        self.clases = numpy.unique(y)
        self.n_clases = self.clases.size
        self.n_atr = X.shape[-1]
        n = X.shape[0]
        self.qa = numpy.zeros((self.n_clases, self.n_atr))
        self.qc = numpy.zeros((self.n_clases))
        for i,c in enumerate(self.clases):
            Xc = X[numpy.where(y == c)]
            nc = Xc.shape[0]
            cuentas = numpy.count_nonzero(Xc, axis = 0)
            self.qa[i, :] = cuentas / nc
            self.qc[i] = nc / n
        # Para evitar logarítmos de zero en cálculo
        # de a posteriori
        self.qa[self.qa == 0] = numpy.nextafter(0, 1)
        self.qa[self.qa == 1] = numpy.nextafter(1, 0)
    
    ## Calcula a posteriori (proporcional) de un conjunto de datos
    def predict_proba(self, X):
        prop = numpy.zeros((X.shape[0], self.n_clases))
        for i in range(self.n_clases):
            a0log = (1 - X) @ numpy.log(1 - self.qa[i, :])
            a1log = X @ numpy.log(self.qa[i, :])
            prop[:, i] = a0log + a1log + numpy.log(self.qc[i])
        return prop

    ## Predice clase de conjunto de datos
    def predict(self, X):
        return numpy.argmax(self.predict_proba(X), axis=1)

In [6]:
## Transforma una matriz de números a otra que tenga atributos Bernoulli
# coloca 1 si encuentra un número diferente de 0
# coloca 0 en otro caso
def transforma_bernoulli(matriz):
    salida = numpy.zeros(matriz.shape)
    for i in range(matriz.shape[0]):
        for j in range(matriz.shape[1]):
            if matriz[i][j] > 0:
                salida[i][j] = 1
    return salida

Se separan los datos entre $x$ (que indican la presencia de palabras) y $y$ (que indican si es spam). También se transforma la matriz de $x$'s en un tipo Bernoulli.

In [7]:
X_ent = data_ent[:, :-1]
y_ent = data_ent[:, -1]
xbern_ent = transforma_bernoulli(X_ent)

Ahora sí, se realiza el entrenamiento del modelo con distribución Bernoulli.

In [8]:
bnbl = BernoulliNBLog()
bnbl.fit(xbern_ent, y_ent)

La otra distribución que se usará será la categórica.

In [9]:
# Clasificador bayesiano ingenuo binario con características categóricas
class CategoricalNB:
    # Calcula probabilidad de la clase
    def prob_clase(self):
        return self.qc
    
    # Calcula probabilidad de atributo dada la clase
    def prob_cond_clase(self, X):
        v = numpy.zeros((X.shape[0], self.clases.size))
        a = numpy.arange(X.shape[1])
        for k in range(X.shape[0]):
            for i in range(self.clases.size):
                v[k, i] = self.qa[i, a, X[k, a]].sum()
        return v

    # Entrena clasificador bayesiano ingenuo
    def fit(self, X, y, alfa = 2):
        # Calcula parámetros de distribución a priori (categórica)
        n = X.shape[0]
        self.clases = numpy.unique(y)
        self.n_clases = self.clases.size
        self.qc = numpy.zeros(self.clases.size)
        for i,c in enumerate(self.clases):
            # Misma alfa para todas las categorías
            self.qc[i] = (numpy.sum(y == c) + alfa - 1) / (n + alfa * self.n_clases - self.n_clases)
        # Escala logarítmica para parámetros de a priori
        self.qc[self.qc == 0] = numpy.nextafter(0, 1)
        self.qc[self.qc == 1] = numpy.nextafter(1, 0)
        self.qc = numpy.log(self.qc)
        # Calcula parámetros de verosimilitud (categórica)
        self.n_atrib = X.shape[1]
        # Presupone mismas categorías en los dos atributos
        self.n_cat = int(numpy.max(X)) + 1
        self.qa = numpy.zeros((self.n_clases, self.n_atrib, self.n_cat))
        # Estima parámetros para cada clase, atributo y categoría (máximo a posteriori)
        for c in self.clases:
            for i in range(self.n_atrib):
                X_ci = X[numpy.where(y == c), i]
                for j in range(self.n_cat): 
                    # Misma alfa para todas las categorías
                    self.qa[c, i, j] = (numpy.sum(X_ci == j) + alfa - 1) / (X_ci.shape[0] + alfa * self.n_cat - self.n_cat)
        # Usa escala logarítmica para parámetros de verosimilitud
        self.qa[self.qa == 0] = numpy.nextafter(0, 1)
        self.qa[self.qa == 1] = numpy.nextafter(1, 0)
        self.qa = numpy.log(self.qa)
    
    # Predice clases dada un conjunto de datos
    def predict(self, X):
        aposteriori = self.prob_clase() + self.prob_cond_clase(X)
        return numpy.argmax(aposteriori, axis = 1)

Ahora se entrena el clasificador

In [10]:
cnb = CategoricalNB()
cnb.fit(X_ent, y_ent)

<h3>Predicciones<h3>

Ya se tienen los dos modelos entrenados. Ahora se harán las predicciones, primero para el de Bernoulli y luego para la categórica.

In [11]:
print("Predicción con Bernoulli")
X_val = data_val[:, :-1]
y_val = data_val[:, -1]
predict_ent = bnbl.predict(xbern_ent)
aciertos_ent = 0
for i in range(predict_ent.shape[0]):
    if predict_ent[i] == y_ent[i]:
        aciertos_ent += 1
fae_bern = aciertos_ent/y_ent.shape[0]
print("Exactitud con datos de entrenamiento:", str(fae_bern*100)+'%')

xbern_val = transforma_bernoulli(X_val)
predict_val = bnbl.predict(xbern_val)
aciertos_val = 0
for i in range(predict_val.shape[0]):
    if predict_val[i] == y_val[i]:
        aciertos_val += 1
fav_bern = aciertos_val/y_val.shape[0]
print("Exactitud con datos de validación:", str(fav_bern*100)+'%')

Predicción con Bernoulli
Exactitud con datos de entrenamiento: 94.65033838221076%
Exactitud con datos de validación: 93.6231884057971%


In [13]:
print("Predicción con categórica")
predict_ent = cnb.predict(X_ent)
aciertos_ent = 0
for i in range(predict_ent.shape[0]):
    if predict_ent[i] == y_ent[i]:
        aciertos_ent += 1
fae_cat = aciertos_ent/y_ent.shape[0]
print("Exactitud con datos de entrenamiento:", str(fae_cat*100)+'%')

predict_val = cnb.predict(X_val)
aciertos_val = 0
for i in range(predict_val.shape[0]):
    if predict_val[i] == y_val[i]:
        aciertos_val += 1
fav_cat = aciertos_val/y_val.shape[0]
print("Exactitud con datos de validación:", str(fav_cat*100)+'%')

Predicción con categórica
Exactitud con datos de entrenamiento: 70.6413148565904%
Exactitud con datos de validación: 72.17391304347827%


La exactitud fue mayor cuando se consideraron los atributos con distribución Bernoulli; es decir, cuando se tomó en cuenta solamente la presencia o ausencia de cierta palabra en el texto y no se contó su frecuencia.

Ahora se tomará el clasificador con distribución Bernoulli para hacer predicciones en el conjunto de datos de prueba.

<h3>Conjunto de datos de prueba</h3>

In [16]:
X_pru = data_pru[:, :-1]
y_pru = data_pru[:, -1]
xbern_pru = transforma_bernoulli(X_pru)
predict_pru = bnbl.predict(xbern_pru)
aciertos_pru = 0
for i in range(predict_pru.shape[0]):
    if predict_pru[i] == y_pru[i]:
        aciertos_pru += 1
fap_bern = aciertos_pru/y_pru.shape[0]
print("Exactitud con datos de prueba:", str(fap_bern*100)+'%')

Exactitud con datos de prueba: 92.15876089060987%
